In [1]:
import torch
import pandas as pd
from rouge import Rouge
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("../Data/hindi_train.csv")
test_data = pd.read_csv("../Data/HindiNews_test.csv")
data

,Id,Heading,Summary,Article
0,hindi_2023_train_0,"गला दबाकर हत्या की; बॉडी बोरे में भरी, लोकल मा...",Kerala Minor Girl Rape Case - केरल के एर्नाकुल...,केरल के एर्नाकुलम जिले में 5 साल की बच्ची से र...
1,hindi_2023_train_1,तेलंगाना में 18 की जान गई; जम्मू-कश्मीर में बा...,इस साल मानसून सीजन में कई राज्यों में भारी तबा...,मानसून सीजन में हुई भारी बारिश ने कई राज्यों म...
2,hindi_2023_train_2,"राजस्थान सरकार बनाएगी कर्ज राहत आयोग, कोर्ट के...",चुनावी साल में राजस्थान सरकार किसानों को लुभान...,चुनावी साल में राजस्थान सरकार किसानों को लुभान...
3,hindi_2023_train_3,"3 से 7 अगस्त तक कर सकेंगे अप्लाय, प्राइस बैंड ...",Non-banking lender SBFC Finance's initial publ...,नॉन बैंकिग फाइनेंस कंपनी 'SBFC फाइनेंस लिमिटेड...
4,hindi_2023_train_4,"डाइनिंग टेबल पर कुकर-कड़ाही न सजाएं, चीन के खा...",स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...,स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...
...,...,...,...,...
21220,hindi_2022_11363,"MSP पर प्रधानमंत्री ने कमेटी बनाई, पराली जलाना...","कृषि मंत्री ने कहा, तीनों कृषि कानूनों को वापस...",नई दिल्ली: कृषि मंत्री नरेंद्र सिंह तोमर ने एम...
21221,hindi_2022_11364,"योगी सरकार ने 9 गेस्ट हाउस का नाम नदियों, धार्...",मुंबई में स्टेट गेस्ट हाउस अब यूपी स्टेट गेस्ट...,लखनऊ: उत्तर प्रदेश की योगी आदित्यनाथ सरकार ने ...
21222,hindi_2022_11365,"Coronavirus से कुछ राहत, 7 लाख के नीचे आए एक्ट...",केंद्रीय स्वास्थ्य मंत्रालय की तरफ से शुक्रवार...,नई दिल्ली। कोरोना वायरस को लेकर कुछ राहत की बा...
21223,hindi_2022_11366,"वाराणसी गंगा नदी के किनारे सबसे साफ शहर बना, प...",लोकसभा में इस शहर का प्रतिनिधित्व करने वाले प्...,नई दिल्ली: केंद्रीय शहरी विकास मंत्री हरदीप सि...


In [3]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(data, test_size=0.15, random_state=42)
print("Training Set:")
print(train_data.shape)
print("Validation Set:")
print(val_data.shape)
print("Test Set:")
print(test_data.shape)

Training Set:
(18041, 4)
Validation Set:
(3184, 4)
Test Set:
(3000, 3)


In [4]:
import re

def preprocess_tokenize(text):
    # for removing punctuation from sentencesc
    text = str(text)
    # text = re.sub(r'(\d+)', r'', text) # remove numbers
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace('\t', ' ')
    text = text.replace('\u200d', '') #
    text = re.sub("(__+)", ' ', str(text)).lower()   #remove _ if it occors more than one time consecutively
    text = re.sub("(--+)", ' ', str(text)).lower()   #remove - if it occors more than one time consecutively
    text = re.sub("(~~+)", ' ', str(text)).lower()   #remove ~ if it occors more than one time consecutively
    text = re.sub("(\+\++)", ' ', str(text)).lower()   #remove + if it occors more than one time consecutively
    text = re.sub("(\.\.+)", ' ', str(text)).lower()   #remove . if it occors more than one time consecutively
    text = re.sub(r"[&©@#ø;.~*!]", ' ', str(text)).lower() #remove <>()|&©ø"',;?~*!
    # text = re.sub(r"[‘’।:]", " ", str(text)) #removing other special characters
    # text = re.sub("([a-zA-Z])",' ',str(text)).lower() #removing all the single characters
    text = re.sub("(\s+)",' ',str(text)).lower() #removing extra spaces
    return text

In [5]:
train_text = [preprocess_tokenize(text) for text in train_data['Article']]
train_summary = [preprocess_tokenize(text) for text in train_data['Summary']]
train_heading = [preprocess_tokenize(text) for text in train_data['Heading']]
val_text = [preprocess_tokenize(text) for text in val_data['Article']]
val_summary = [preprocess_tokenize(text) for text in val_data['Summary']]
val_heading = [preprocess_tokenize(text) for text in val_data['Heading']]
test_heading = [preprocess_tokenize(text) for text in test_data['Heading']]
test_text = [preprocess_tokenize(text) for text in test_data['Article']]

In [6]:
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class FFTActivation(nn.Module):
    def __init__(self):
        super(FFTActivation, self).__init__()
        self.fft = torch.fft.rfft

    def forward(self, x):
        return self.fft(x)

class SummarizartionModel(nn.Module):
    def __init__(self, model, tokenizer):
        super(SummarizartionModel, self).__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.fft = FFTActivation()

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None):
        out = self.model(input_ids=input_ids, labels=decoder_input_ids, 
                attention_mask=attention_mask, decoder_attention_mask=decoder_attention_mask)
        out.encoder_last_hidden_state = self.fft(out.encoder_last_hidden_state)
        out.logits = self.fft(out.logits)
        return out
        

    def generate(self, text, max_length=50):
        input_ids = self.tokenizer(text, return_tensors='pt', truncation=True).input_ids.to(device)
        attention_mask = self.tokenizer(text, return_tensors='pt', truncation=True).attention_mask.to(device)
        summary_ids = self.model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, num_beams=2, length_penalty=0.5, no_repeat_ngram_size=2, early_stopping=True)
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

def generate_summary(summary_model, tokenizer, text, max_length):
    input_ids = tokenizer(text, return_tensors='pt', max_length=1024, truncation=True).to(device)
    summary_ids = summary_model.model.generate(input_ids.input_ids, attention_mask=input_ids.attention_mask, 
                                        max_length=max_length, num_beams=2, length_penalty=0.5, 
                                        no_repeat_ngram_size=2, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary
    

In [7]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "csebuetnlp/mT5_m2o_hindi_crossSum"
tokenizer_new = AutoTokenizer.from_pretrained(model_name)
model_new = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

loaded_model = SummarizartionModel(model_new, tokenizer_new)
loaded_model.load_state_dict(torch.load("summary_model_mT5_m2o.pth"))
loaded_model = loaded_model.to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
ind = 0
text = val_text[ind]
heading = val_heading[ind]
summary = val_summary[ind]
# text = heading + " " + text

generated_summary = generate_summary(loaded_model, tokenizer_new, text, max_length=100)
# generated_summary = loaded_model.generate(text, max_length=100)

print(f"Text: {text}")
print(f"Heading: {heading}")
print(f"Summary: {summary}")
print(f"Generated Summary: {generated_summary}")

Text: मथुरा: उत्तर प्रदेश के मथुरा जिले में यमुना एक्सप्रेस-वे पर आधा दर्जन बदमाशों ने दिल्ली से हमीरपुर जा रही निजी बस में सवारी के रूप में चढ़कर उसे बंधक बना लिया और तकरीबन सभी सवारियों से लाखों रुपये का सामान लूट किया। घटना की जानकारी मिलने पर आगरा रेंज के पुलिस महानिरीक्षक समेत तमाम वरिष्ठ अधिकारी मौके पर पहुंचे। पुलिस बदमाशों की खोज में जुट गई है। पुलिस के मुताबिक, लूट की घटना उस समय घटी जब बीती रात दिल्ली से हमीरपुर के लिए निकली निजी बस करीब एक बजे नोएडा क्षेत्र से आगे मथुरा जिले की सुरीर कोतवाली क्षेत्र में पहुंची। पुलिस के मुताबिक, बदमाश सवारियों के रूप में बस में चढ़े थे और उन्होंने कुछ समय के बाद ही हथियारों के बल पर बस को बंधक बना लिया और सवारियों से नकदी व जेवर लूट लिए। घटना को अंजाम देने के बाद बदमाश जब बस से उतरकर फरार हो गए तो चालक ने पुलिस को सूचना दी। बस लूट की जानकारी मिलने पर आईजी नवीन अरोड़ा, एसएसपी डॉ गौरव ग्रोवर मौके पर पहुंच गए। एसएसपी डॉ गौरव ग्रोवर ने बताया कि बस की सवारियों से बदमाशों का हुलिया, बोली, चाल-ढाल आदि जानकारी जुटाने के बाद कई अलग-अलग टीमें उनकी तलाश म

In [12]:
from tqdm import tqdm

def generate_summaries(text, heading, tokenizer, model, data_name):
    generated_summary = []
    model = model.to(device)

    for i, input_text in tqdm(enumerate(text), total=len(text), desc="Processing Texts"):
        # summary = model.generate('summarize:' + heading[i] + " " + text[i], max_length=80)
        # input_text = 'summarize:' + text[i]
        input_text = 'summarize:' + heading[i] + " " + text[i]
        summary = generate_summary(model, tokenizer, input_text, max_length=100)
        generated_summary.append(summary)
    print('summary generated for ', data_name)
    return generated_summary

val_generated_summary = generate_summaries(test_text, test_heading, tokenizer_new, loaded_model, "Test Data") 

Processing Texts: 100%|██████████| 3000/3000 [1:28:04<00:00,  1.76s/it]

summary generated for  Test Data


In [14]:
test_data['Summary'] = val_generated_summary
test_data.to_csv("../Data/HindiNews_test__mT5_m2o.csv", index=False)